In [1]:
import requests
import rich
%load_ext rich
from rich import print
from gzip import decompress
import json
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import zstandard

In [2]:
r=requests.get("https://yaqwsx.github.io/jlcparts/data/index.json")
categories=r.json()["categories"]
categories


{
    'ADC/DAC/Data Conversion': {
        'ADC/DAC - Specialized': {
            'datahash': 'ae7b36c82e22edc6cffcde317ab2e62c94a8ecd4a93a2b64f4c6bd36301f2683',
            'sourcename': 'ADCakaDACakaData_ConversionADCakaDAC___Specialized',
            'stockhash': '17dd09c3c06f7042db452fa5ac81ca340597d436cd18edfe2458644fa1f5050a'
        },
        'Analog Front End (AFEs)': {
            'datahash': '2da274879929877997164ca5dd39702428c86b750fb7e6bd289d0c753bf38816',
            'sourcename': 'ADCakaDACakaData_ConversionAnalog_Front_End__AFEs_',
            'stockhash': 'df73afb6ae86d93d5868a84c7327079d41898343302569393534a0f8ad4280d2'
        },
        'Analog To Digital Converters (ADCs)': {
            'datahash': '4c602424451d95c14281044ae00cac0e08defbb52e5396a57c7d9ebd1344f685',
            'sourcename': 'ADCakaDACakaData_ConversionAnalog_To_Digital_Converters__ADCs_',
            'stockhash': '91d8e3ad4645f6296355adb91436b66e9a74c30f41d7c6c50b58aa87d76e15b7'
        },
      

In [31]:

f = widgets.IntProgress(min=0, max=len(categories)) # instantiate the bar
display(f) # display the bar
inner_bar = widgets.IntProgress(min=0, max=10) # instantiate the bar
display(inner_bar) # display the bar
output=widgets.Output(layout={'border': '1px solid black'})
display(output)
@output.capture()

def fetch_data():
    components=pd.DataFrame()
    global f, inner_bar, output
    for cat, subcats in categories.items():
        inner_bar.max=len(subcats)
        inner_bar.value=0
        inner_bar.description=cat
        for subcat_name, details in subcats.items():
            output.clear_output(wait=True)
            while True:
                try:
                    
                    print(subcat_name)
                    r=requests.get("https://yaqwsx.github.io/jlcparts/data/"+details["sourcename"]+".json.gz",timeout=10)
                    subcat_dict=json.loads(decompress(r.content))
                    subcat=pd.DataFrame(columns=subcat_dict["schema"]+["category","subcategory"])
                    subcat[subcat_dict["schema"]]=np.asarray(subcat_dict["components"],dtype="object")
                    subcat["subcategory"]=subcat_name
                    subcat["category"]=cat
                    
                    r=requests.get("https://yaqwsx.github.io/jlcparts/data/"+details["sourcename"]+".stock.json",timeout=10)
                    catstock=pd.DataFrame(json.loads(r.content).items(),columns=["lcsc","stock"])
                    subcat=pd.merge(subcat,catstock, on="lcsc")
                    if components.empty:
                        components=subcat
                    else:
                        components=pd.concat([components,subcat])
                    inner_bar.value+=1
                except requests.exceptions.Timeout:
                    continue
                break
        f.value+=1
    return components

        
components= fetch_data()
components.to_pickle("cache/components.pickle",compression="zstd")
components
#print(stock)


       

IntProgress(value=0, max=113)

IntProgress(value=0, max=10)

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

,lcsc,mfr,joints,description,datasheet,price,img,url,attributes,category,subcategory,stock
0,C239896,AD421BRZRL7,5,16 SOIC-16-300mil ADCs/DACs - Special Purpose...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 7.49, 'qFrom': 1, 'qTo': 9}, {'pric...",20211009_Analog-Devices-AD421BRZRL7_C239896_fr...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,3
1,C459871,AD2S1210WDSTZRL7,48,4.75V~5.25V LQFP-48(7x7) ADCs/DACs - Special ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 8.491428571, 'qFrom': 1, 'qTo': 9},...",20230106_Analog-Devices-AD2S1210WDSTZRL7_C4598...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,126
2,C578328,AD598JRZ,20,SOIC-20 ADC/DAC - Specialized ROHS,https://datasheet.lcsc.com/lcsc/2204121100_Ana...,"[{'price': 67.968571429, 'qFrom': 1, 'qTo': 2}...",20230106_Analog-Devices-AD598JRZ_C578328_back.jpg,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,41
3,C649048,AD1851RZ-REEL7,2,5V 16 PCM SOIC-16-300mil ADCs/DACs - Special ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 20.268571429, 'qFrom': 1, 'qTo': 2}...",20230130_Analog-Devices-AD1851RZ-REEL7_C649048...,ADC-DAC---Specialized_Analog-Devices-Analog-De...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,100
4,C649721,AD5412ACPZ-REEL7,2,±1LSB 2.7V~5.5V 12 Bits LFCSP-40(6x6) Digital...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 12.705714286, 'qFrom': 1, 'qTo': 9}...",None,Digital-To-Analog-Converters-DACs_Analog-Devic...,"{'Basic/Extended': {'format': '${default}', 'p...",ADC/DAC/Data Conversion,ADC/DAC - Specialized,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9,C6461551,61300511121,5,3A Straight 250V 5P -40℃~+105℃ 2.54mm Black Pl...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 0.250571429, 'qFrom': 2, 'qTo': 98}...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,1137
10,C6620841,FTSH-105-01-F-DV,10,3.4A Tin Standing paste 10P -55℃~+125℃ 1.27mm ...,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 1.355714286, 'qFrom': 1, 'qTo': 9},...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,1
11,C6968485,IFX007TAUMA1,8,TO-263-7-1 Gate Drivers ROHS,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 2.14, 'qFrom': 1, 'qTo': 9}, {'pric...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,61
12,C7125816,TPSM63610RDFR,22,B3QFN-22(6.5x7.5) DC-DC Power Modules ROHS,https://wmsc.lcsc.com/wmsc/upload/file/pdf/v2/...,"[{'price': 18.871428571, 'qFrom': 1, 'qTo': 9}...",None,None,"{'Basic/Extended': {'format': '${default}', 'p...",others,others,15


In [3]:
components=pd.read_pickle("cache/components.pickle",compression="zstd")


In [16]:
a=components["attributes"].map(lambda x:[x["values"] for x in x.values()]) #.list.flatten()

In [47]:
a.map(lambda y:max([len(x.keys()) for x in y])>1)


0     False
1     False
2     False
3     False
4     False
      ...  
9     False
10    False
11    False
12    False
13    False
Name: attributes, Length: 340976, dtype: bool

In [53]:
a[a.map(lambda y:max([len(x.keys()) for x in y])>1)]#.list.flatten().unique()


0       [{'default': ['圆形电缆', 'string']}, {'default': ...
1       [{'default': ['圆形电缆', 'string']}, {'default': ...
7       [{'default': ['圆形电缆', 'string']}, {'default': ...
2       [{'default': ['Extended', 'string']}, {'defaul...
4       [{'default': ['Extended', 'string']}, {'defaul...
                              ...                        
4003    [{'default': ['Extended', 'string']}, {'defaul...
1       [{'default': ['Extended', 'string']}, {'defaul...
49      [{'default': ['Extended', 'string']}, {'defaul...
106     [{'default': ['Extended', 'string']}, {'defaul...
109     [{'default': ['Extended', 'string']}, {'defaul...
Name: attributes, Length: 26623, dtype: object

In [ ]:
output=list()
for cat, subcats in categories.items():
    for subcat, details in subcats.items():
        with open('cache/'+details["sourcename"]+".json.gz", 'rb') as f:
            with open('cache/'+details["sourcename"]+".stock.json", 'rb') as stockf:
                
                r=f.read()
                components=json.loads(decompress(r))["components"]
                stock=json.loads(stockf.read())
                basic=[x for x in components if x[8]['Basic/Extended']["values"]["default"][0]!="Extended"]
                for x in basic:
                    elem={
                        "category":subcat,
                        "part_number": x[0],
                        "man_part_number": x[1],
                        "description":x[3],
                        "price": x[5],
                        "basic": x[8]['Basic/Extended']["values"]["default"][0],
                        "manufacturer": x[8]['Manufacturer']["values"]["default"][0],
                        "package": x[8]['Package']["values"]["default"][0],
                        "datasheet": x[4],
                        "stock": stock[x[0]]
                    }
                    output.append(elem)


In [ ]:
df=pd.DataFrame(output)
df[df["part_number"]=="C2869803"]


In [74]:
list(json.loads(r.content).items())

[('C5349995', 0), ('C578360', 15), ('C578504', 2), ('C578505', 482)]